In [152]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import pandas as pd
import ast
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, ConfusionMatrixDisplay

# Chargement du modèle français pré-entraîné de spaCy
nlp_fr = spacy.load("fr_core_news_md")

# Fonction de test NER sur un texte de démonstration
def test_ner_on_text(text, model, labels=None, options=None):
    doc = model(text)
    spacy.displacy.render(doc, style="ent", options=options, jupyter=True)
    
# Exemple de texte pour tester le modèle avant et après l'entraînement
example_text = '''Je voudrais aller de Toulouse à Bordeaux.
Comment me rendre à Port-Boulet depuis la gare de Tours ?
Je veux aller voir mon ami Albert à Tours en partant de Bordeaux.'''

# Tester le modèle de base
print("Test NER avec le modèle de base:")
test_ner_on_text(example_text, nlp_fr)

### Chargement des données à partir d'un CSV et préparation pour l'entraînement
# Lecture des données annotées
df = pd.read_csv('../../dataset/raw/initial_training_data.csv', 
                 sep=',', 
                 encoding='utf-8', 
                 on_bad_lines='skip')

# Aperçu rapide des données
df.head()

# Préparation des données pour spaCy
def prepare_spacy_data(df, nlp_model, output_path):
    db = DocBin()  # Conteneur pour les documents annotés

    data = list(zip(df['text'], df['entities']))
    shuffle(data)  # Mélange des données

    def verify_entities(text, entities):
        verified = []
        for ent in entities:
            start, end, label = ent['start'], ent['end'], ent['label']
            entity_text = text[start:end]
            # Vérification d'alignement des entités
            if entity_text == text[start:end]:
                verified.append(ent)
        return verified

    for text, entities in data:
        entities = ast.literal_eval(entities)  # Conversion des entités du format string au format dict
        verified_entities = verify_entities(text, entities)
        if verified_entities:
            doc = nlp_model.make_doc(text)
            ents = []
            for ent in verified_entities:
                start, end, label = ent['start'], ent['end'], ent['label']
                span = doc.char_span(start, end, label=label, alignment_mode="contract")
                if span:
                    ents.append(span)
            doc.ents = ents
            db.add(doc)

    # Sauvegarde des données dans un format compatible avec spaCy
    db.to_disk(output_path)

# Préparation des données d'entraînement
train_data_path = "../../dataset/processed/processed_training_data.spacy"
prepare_spacy_data(df, nlp_fr, train_data_path)

### Entrainement du modèle personnalisé
# Création de fichier de configuration
!python -m spacy init config ./config_itineraire.cfg --lang fr --pipeline ner --optimize efficiency --force

# Lancement de l'entraînement
!python -m spacy train ./config_itineraire.cfg --output ../../models/saved_models --paths.train {train_data_path} --paths.dev {train_data_path}

### Évaluation et visualisation des résultats

# Chargement du modèle entraîné
nlp_custom = spacy.load("../../models/saved_models/model-best/")

# Définir les couleurs pour les entités
colors = {"DEPARTURE": "#ffe899", "DESTINATION": "#b1ff5e", "ESCALE": "#82b8ff"}
options = {"ents": ["DEPARTURE", "DESTINATION", "ESCALE"], "colors": colors}

# Test sur le texte simple
print("Test NER avec le modèle personnalisé:")
test_ner_on_text(example_text, nlp_custom, options=options)

# Texte plus complexe pour tester davantage
complex_text = '''Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence.
Je compte me rendre à Bordeaux depuis Marseille pour rendre visite à ma soeur Paris.'''

print("Test sur un texte plus complexe:")
test_ner_on_text(complex_text, nlp_custom, options=options)

### Évaluation du modèle avec des graphiques

# Exemple de texte avec des escales
text_with_stops = '''Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence avec une escale à Nice.
Je compte me rendre à Bordeaux depuis Marseille en m'arrêtant à Toulouse pour rendre visite à ma soeur Paris.'''

# Fonction pour afficher des métriques de classification
def evaluate_model(texts, labels, model):
    preds = []
    trues = []
    
    for text, true_labels in zip(texts, labels):
        doc = model(text)
        pred_labels = [(ent.start_char, ent.end_char, ent.label_) for ent in doc.ents]
        preds.extend(pred_labels)
        trues.extend(true_labels)
    
    return classification_report(trues, preds, output_dict=True), trues, preds

# Générer des prédictions et afficher un rapport de classification
texts = df['text'].tolist()
entities = [ast.literal_eval(e) for e in df['entities'].tolist()]

metrics, true_labels, pred_labels = evaluate_model(texts, entities, nlp_custom)

# Afficher un rapport de classification
print("Rapport de classification:")
print(classification_report(true_labels, pred_labels))

# Visualiser la matrice de confusion pour les prédictions
ConfusionMatrixDisplay.from_predictions(
    [label[2] for label in true_labels], 
    [label[2] for label in pred_labels], 
    cmap="Blues"
)
plt.title("Matrice de Confusion des Entités")
plt.show()

# Visualisation des métriques avec des graphiques
def plot_metrics(metrics):
    df_metrics = pd.DataFrame(metrics).transpose()
    df_metrics = df_metrics.drop(columns=["support"])

    plt.figure(figsize=(10, 6))
    sns.heatmap(df_metrics, annot=True, fmt=".2f", cmap="YlGnBu")
    plt.title("Rapport de Classification par Catégorie")
    plt.show()

plot_metrics(metrics)

Test NER avec le modèle de base:


AttributeError: 'NoneType' object has no attribute 'get'